In [ ]:
# The usual preamble
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Make the graphs a bit prettier, and bigger
#pd.set_option('display.mpl_style', 'default')
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['font.family'] = 'sans-serif'

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

We're going to use a new dataset here, to demonstrate how to deal with larger datasets. This is a subset of the of 311 service requests from [NYC Open Data](https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9). 

In [ ]:
#load 311 dataset
pd.read_csv("311-service-requests.csv")

Depending on your pandas version, you might see an error like "DtypeWarning: Columns (8) have mixed types". This means that it's encountered a problem reading in our data. In this case it almost certainly means that it has columns where some of the entries are strings and some are integers.

For now we're going to ignore it and hope we don't run into a problem, but in the long run we'd need to investigate this warning.

# 2.1 What's even in it? (the summary)

When you print a large dataframe, it will only show you the first few rows.

If you don't see this, don't panic! The default behavior for large dataframes changed between pandas 0.12 and 0.13. Previous to 0.13 it would show you a summary of the dataframe. This includes all the columns, and how many non-null values there are in each column.

In [ ]:
#show content


# 2.2 Selecting columns and rows

To select a column, we index with the name of the column, like this:

In [ ]:
#Selecting the Complaint Type


To get the first 5 rows of a dataframe, we can use a slice: `df[:5]`.

This is a great way to get a sense for what kind of information is in the dataframe -- take a minute to look at the contents and get a feel for this dataset.

In [ ]:
#show first 5 rows


We can combine these to get the first 5 rows of a column:

In [ ]:
#show first 5 rows of complaint column


and it doesn't matter which direction we do it in:

In [ ]:
#switch order of showing


# 2.3 Selecting multiple columns

What if we just want to know the complaint type and the borough, but not the rest of the information? Pandas makes it really easy to select a subset of the columns: just index with list of columns you want.

In [ ]:
#Show complaint type and Borough


That showed us a summary, and then we can look at the first 10 rows:

In [ ]:
# Complaint Type and Borough and first 10 rows

# 2.4 What's the most common complaint type?

This is a really easy question to answer! There's a `.value_counts()` method that we can use:

In [ ]:
#Show most common complaint type

If we just wanted the top 10 most common complaints, we can do this:

In [ ]:
#show top 10
#Show most common complaint type


But it gets better! We can plot them!

In [ ]:
#use plot function

# Selecting only noise complaints

Let's continue with our NYC 311 service requests example.

In [5]:
complaints = pd.read_csv('311-service-requests.csv')

I'd like to know which borough has the most noise complaints. Again, first, we'll take a look at the data to see what it looks like:

To get the noise complaints, we need to find the rows where the "Complaint Type" column is "Noise - Street/Sidewalk". 

In [ ]:
# select borough with most noise complaints

This is a big array of `True`s and `False`s, one for each row in our dataframe. When we index our dataframe with this array, we get just the rows where our boolean array evaluated to `True`.  It's important to note that for row filtering by a boolean array the length of our dataframe's index must be the same length as the boolean array used for filtering.

You can also combine more than one condition with the `&` operator like this:

Combine your query, Select all noise - complaints, where Borough equals Brooklyn 

In [ ]:
# Noise complaints where Borough = Brooklyn


Just select a few necessary columns

In [ ]:
# Show output for following columns
# Complaint Type, Borough, Created Date, Descriptor


# So, which borough has the most noise complaints?

In [ ]:
# Which borough has the most noise complaints?
# List all nose complaints per borough


But what if we wanted to divide by the total number of complaints, to make it make a bit more sense? That would be easy too:

In [ ]:
#Normalize your data


Plot neighbourhood with most complaints

Which one it is?

# How do we know if it's messy? 

One of the main problems with messy data is: how do you know if it's messy or not?

We're going to use the NYC 311 service request dataset again here, since it's big and a bit unwieldy.

In [6]:
requests = pd.read_csv('311-service-requests.csv')

We're going to look at a few columns here. I know already that there are some problems with the zip code, so let's look at that first.
 
To get a sense for whether a column has problems, I usually use `.unique()` to look at all its values. If it's a numeric column, I'll instead plot a histogram to get a sense of the distribution.

When we look at the unique values in "Incident Zip", it quickly becomes clear that this is a mess.

Some of the problems:

* Some have been parsed as strings, and some as floats
* There are `nan`s 
* Some of the zip codes are `29616-0759` or `83`
* There are some N/A values that pandas didn't recognize, like 'N/A' and 'NO CLUE'

What we can do:

* Normalize 'N/A' and 'NO CLUE' into regular nan values
* Look at what's up with the 83, and decide what to do
* Make everything strings

In [7]:
# take a deep look at your data

In [ ]:
#Show all Zips, grouped


# Fixing the nan values and string/float confusion

We can pass a `na_values` option to `pd.read_csv` to clean this up a little bit. We can also specify that the type of Incident Zip is a string, not a float.

In [ ]:
# specifiy how to handle na_values and how to handle the Incidient Zip (In this case as a string)


Display your output once again

# What's up with the dashes?

Get all rows with dashes

In [ ]:
# Write code to retrieve rows with dashes


Display our rows with dashes

In [ ]:
# display rows with dashes


I thought these were missing data and originally deleted them like this:

`requests['Incident Zip'][rows_with_dashes] = np.nan`

But then my friend Google pointed out that 9-digit zip codes are normal. Let's look at all the zip codes with more than 5 digits, make sure they're okay, and then truncate them.

In [ ]:
#show all zip codes greater than 5 / group them

Those all look okay to truncate to me.

In [ ]:
#truncate those zips, slice the first part and store it back to our incident zip column

### Done.

Earlier I thought 00083 was a broken zip code, but turns out Central Park's zip code 00083! Shows what I know. I'm still concerned about the 00000 zip codes, though: let's look at that. 

In [ ]:
#Show all requests with zip starting 00000


This looks bad to me. Let's set these to nan.

In [ ]:
#set all zips with 00000 to nan


Great. Let's see where we are now:

Amazing! This is much cleaner. There's something a bit weird here, though -- I ask you to look up 77056 on Google maps?

Let's take a closer look:

In [ ]:
# Let's say the zips starting with '0' and '1' are okay, for now. (this isn't actually true -- 13221 is in Syracuse, and why?)
# is_close = zips that are close, starting with 0 and 1
# There are a bunch of NaNs, but we're not interested in them right now, so we'll say they're False
# is_far = not is close AND zips that are not null

In [ ]:
# show all far requests and their incident zip, descriptor and city. 
#Sort it by Incident Zip


Okay, there really are requests coming from LA and Houston! Good to know. Filtering by zip code is probably a bad way to handle this -- we should really be looking at the city instead.

In [ ]:
#Filter all requesty by the city they're coming from


It looks like these are legitimate complaints, so we'll just leave them alone.

# Putting it together

Here's what we ended up doing to clean up our zip codes, all together:

In [ ]:
#write a function which sclices away the unnecesary part

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  